        Note:
                Before running the notebook download and extract the complaints.csv zip file, the link for downloading zip file given in the 'docs file' and then read the file in this notebook
                You can see the numbers on all graph plots just by hovering over it
                Please ignore the dependencies error like ValueError


# __Q1__


        Data set reference link: https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data
        File data source: https://files.consumerfinance.gov/ccdb/complaints.csv.zip 
        Problem statement:
                Download the data from the file data source and provide possible data insights.


        Importing Required Libraries

In [23]:
! pip install pandas
! pip install plotly
import pandas as pd
import numpy as np
import plotly.express as px


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


        Reading complaints.csv dataset using pandas and displaying top 30 records

In [3]:
df = pd.read_csv('complaints.csv')
df.head(30)

C:\Users\rames\AppData\Local\Temp\ipykernel_25232\3419974438.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('complaints.csv')


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2024-01-24,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account information incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33578,NaN,Consent not provided,Web,2024-01-24,Closed with explanation,Yes,NaN,8205544
1,2024-02-26,Debt collection,Other debt,Threatened to contact someone or share informa...,Talked to a third-party about your debt,NaN,NaN,"Marlette Funding, LLC",IL,60060,NaN,Consent not provided,Web,2024-02-26,Closed with explanation,Yes,NaN,8424268
2,2024-02-26,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company believes it acted appropriately as aut...,"CCS Financial Services, Inc.",NC,27536,NaN,Consent not provided,Web,2024-02-26,Closed with non-monetary relief,Yes,NaN,8423775
3,2024-02-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,I am disappointed to have received a credit re...,NaN,LEXISNEXIS,CA,94124,NaN,Consent provided,Web,2024-02-20,Closed with explanation,Yes,NaN,8379830
4,2024-01-26,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,"The companies, along with the credit reporting...",NaN,"EQUIFAX, INC.",NC,28804,NaN,Consent provided,Web,2024-01-26,Closed with non-monetary relief,Yes,NaN,8232435
5,2024-02-21,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,CL Holdings LLC,CA,90221,NaN,Consent not provided,Web,2024-02-21,Closed with explanation,Yes,NaN,8381840
6,2024-02-20,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Other transaction problem,NaN,NaN,NaN,"HUNTINGTON NATIONAL BANK, THE",MI,48912,NaN,NaN,Phone,2024-02-20,Closed with monetary relief,Yes,NaN,8385193
7,2024-02-21,Debt collection,Other debt,Written notification about debt,Didn't receive notice of right to dispute,NaN,Company has responded to the consumer and the ...,"HOVG, LLC",NJ,08755,NaN,Consent not provided,Web,2024-02-21,Closed with explanation,Yes,NaN,8392661
8,2024-02-21,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,CL Holdings LLC,VA,23235,NaN,Consent not provided,Web,2024-02-21,Closed with explanation,Yes,NaN,8392962
9,2024-02-21,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,Lexis Nexis provided information to XXXX and X...,NaN,LEXISNEXIS,GA,302XX,NaN,Consent provided,Web,2024-02-21,Closed with explanation,Yes,NaN,8381710


        Verifying data quality

In [14]:
df.dtypes

Date received                   object
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public response         object
Company                         object
State                           object
ZIP code                        object
Tags                            object
Consumer consent provided?      object
Submitted via                   object
Date sent to company            object
Company response to consumer    object
Timely response?                object
Consumer disputed?              object
Complaint ID                     int64
dtype: object

In [6]:
df.isna().sum()

Date received                         0
Product                               0
Sub-product                      235292
Issue                                 2
Sub-issue                        735101
Consumer complaint narrative    3351574
Company public response         2693192
Company                               0
State                             45611
ZIP code                          30225
Tags                            4684932
Consumer consent provided?      1026812
Submitted via                         0
Date sent to company                  0
Company response to consumer         14
Timely response?                      0
Consumer disputed?              4399466
Complaint ID                          0
dtype: int64

In [9]:
missing_percentage = (df.isna().sum() / len(df)) * 100          # Calculating the percentage of missing values for each column

missing_percentage

Date received                    0.000000
Product                          0.000000
Sub-product                      4.553056
Issue                            0.000039
Sub-issue                       14.224691
Consumer complaint narrative    64.855174
Company public response         52.115047
Company                          0.000000
State                            0.882603
ZIP code                         0.584874
Tags                            90.656533
Consumer consent provided?      19.869491
Submitted via                    0.000000
Date sent to company             0.000000
Company response to consumer     0.000271
Timely response?                 0.000000
Consumer disputed?              85.132577
Complaint ID                     0.000000
dtype: float64

        As we can see in the above missing values percentage result,
            Consumer complaint narrative, Company public response, Tags, and Consumer disputed? 
            contains missing values more than 33% of their respective column values

            So I am considering removing them, as scenario given for missing values

### __Data Cleaning and Preprocessing__


In [10]:
# Setting the threshold for non-null values
threshold = len(df) * 0.60  # I'm keeping 40% threshold for keeping the column

# Drop columns with missing data exceeding the threshold
df1 = df.dropna(thresh=threshold, axis=1)

# Display the columns remaining in the cleaned DataFrame
print("Columns remaining after dropping:", df1.columns)


Columns remaining after dropping: Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Company', 'State', 'ZIP code', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Complaint ID'],
      dtype='object')


        Now we have only 14 columns

In [11]:
len(df1.columns)

14

In [16]:
df1.shape

(5167782, 14)

        Converting Date columns to datetime format

In [26]:
df1['Date received'] = pd.to_datetime(df1['Date received'])
df1['Date sent to company'] = pd.to_datetime(df1['Date sent to company'])

C:\Users\rames\AppData\Local\Temp\ipykernel_25232\3116376936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date received'] = pd.to_datetime(df1['Date received'])
C:\Users\rames\AppData\Local\Temp\ipykernel_25232\3116376936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date sent to company'] = pd.to_datetime(df1['Date sent to company'])


        Remove leading and trailing whitespaces from text columns

In [31]:
df1.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Company', 'State', 'ZIP code', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Complaint ID'],
      dtype='object')

In [33]:
text_columns = ['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Company', 'State', 'ZIP code', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Complaint ID']
df1[text_columns] = df1[text_columns].apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

C:\Users\rames\AppData\Local\Temp\ipykernel_25232\2590363827.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[text_columns] = df1[text_columns].apply(lambda x: x.str.strip() if x.dtype == 'object' else x)


In [27]:
df1.dtypes

Date received                   datetime64[ns]
Product                                 object
Sub-product                             object
Issue                                   object
Sub-issue                               object
Company                                 object
State                                   object
ZIP code                                object
Consumer consent provided?              object
Submitted via                           object
Date sent to company            datetime64[ns]
Company response to consumer            object
Timely response?                        object
Complaint ID                             int64
dtype: object

In [28]:
df1.head(20)

,Date received,Product,Sub-product,Issue,Sub-issue,Company,State,ZIP code,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Complaint ID
0,2024-01-24,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account information incorrect,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33578,Consent not provided,Web,2024-01-24,Closed with explanation,Yes,8205544
1,2024-02-26,Debt collection,Other debt,Threatened to contact someone or share informa...,Talked to a third-party about your debt,"Marlette Funding, LLC",IL,60060,Consent not provided,Web,2024-02-26,Closed with explanation,Yes,8424268
2,2024-02-26,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,"CCS Financial Services, Inc.",NC,27536,Consent not provided,Web,2024-02-26,Closed with non-monetary relief,Yes,8423775
3,2024-02-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,LEXISNEXIS,CA,94124,Consent provided,Web,2024-02-20,Closed with explanation,Yes,8379830
4,2024-01-26,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,"EQUIFAX, INC.",NC,28804,Consent provided,Web,2024-01-26,Closed with non-monetary relief,Yes,8232435
5,2024-02-21,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,CL Holdings LLC,CA,90221,Consent not provided,Web,2024-02-21,Closed with explanation,Yes,8381840
6,2024-02-20,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Other transaction problem,Information belongs to someone else,"HUNTINGTON NATIONAL BANK, THE",MI,48912,Consent not provided,Phone,2024-02-20,Closed with monetary relief,Yes,8385193
7,2024-02-21,Debt collection,Other debt,Written notification about debt,Didn't receive notice of right to dispute,"HOVG, LLC",NJ,08755,Consent not provided,Web,2024-02-21,Closed with explanation,Yes,8392661
8,2024-02-21,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,CL Holdings LLC,VA,23235,Consent not provided,Web,2024-02-21,Closed with explanation,Yes,8392962
9,2024-02-21,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,LEXISNEXIS,GA,302XX,Consent provided,Web,2024-02-21,Closed with explanation,Yes,8381710


        Checking Row duplication

In [21]:
df1.duplicated().sum()

0

        Now, to impute missing values for column types 'object', I am imputing by their 'mode'

In [24]:
# Impute missing values for categorical columns with the mode
categorical_cols = df1.select_dtypes(include='object').columns
df1[categorical_cols] = df1[categorical_cols].fillna(df1[categorical_cols].mode().iloc[0])

C:\Users\rames\AppData\Local\Temp\ipykernel_25232\2473308408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[categorical_cols] = df1[categorical_cols].fillna(df1[categorical_cols].mode().iloc[0])


In [25]:
df1.isna().sum()

Date received                   0
Product                         0
Sub-product                     0
Issue                           0
Sub-issue                       0
Company                         0
State                           0
ZIP code                        0
Consumer consent provided?      0
Submitted via                   0
Date sent to company            0
Company response to consumer    0
Timely response?                0
Complaint ID                    0
dtype: int64

        Now, as we can see our data frame is cleaned, 
        As we don't have numeric column, so I have not written the code to impute those
         now we are good to go to find the Insights

### __Insights__

        Insight 1: Total number of complaints

In [34]:
total_complaints = len(df)
total_complaints

5167782

In [37]:
pip install nbformat --upgrade

     ---------------------------------------- 78.5/78.5 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 88.3/88.3 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 209.0/209.0 kB 4.3 MB/s eta 0:00:00
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


         Your can see the result just by hovering over the graph plots

         Please ignore ValueError, that is the error of some dependencies

        # Insight 2:Complaints by date received

In [46]:
complaints_by_date = df1.groupby(df1['Date received']).size()
fig = px.line(complaints_by_date, title='Complaints by Date Received', labels={'index': 'Date', 'value': 'Number of Complaints'})
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Number of Complaints')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=0<br>Date received=%{x}<br>Number of Complaints=%{y}<extra></extra>',
              'legendgroup': '0',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '0',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2011, 12, 1, 0, 0),
                          datetime.datetime(2011, 12, 2, 0, 0),
                          datetime.datetime(2011, 12, 3, 0, 0), ...,
                          datetime.datetime(2024, 5, 4, 0, 0),
                          datetime.datetime(2024, 5, 5, 0, 0),
                          datetime.datetime(2024, 5, 6, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 120,  138,   26, ..., 2859, 1712, 2002], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Complaints by Date Received'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Number of Complaints'}}}
})

        In the above diagram we can see Number of Complaints w.r.t the date where,
        we can see the trend of complaint is increasing with the time, where we can see in 'April 2024' highest number of complaints were reported, with the number as 7747

        # Insight 3: Product by highest complaint volume

In [49]:
product_complaints = df1['Product'].value_counts().reset_index()
product_complaints.columns = ['Product', 'Number of Complaints']
fig = px.bar(product_complaints, x='Product', y='Number of Complaints', title='Product by Highest Complaint Volume')
fig.update_xaxes(title_text='Product')
fig.update_yaxes(title_text='Number of Complaints')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Product=%{x}<br>Number of Complaints=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Credit reporting, credit repair services, or other personal consumer reports',
                          'Credit reporting or other personal consumer reports',
                          'Debt collection', 'Mortgage', 'Checking or savings account',
                          'Credit card or prepaid card', 'Credit reporting', 'Credit card',
                          'Student loan', 'Bank account or service',
                          'Money transfer, virtual currency, or money service',
                          'Vehicle loan or lease', 'Consumer Loan',
                          'Payday loan, title loan, or personal loan', 'Prepaid card',
                          'Payday loan',
                          'Payday loan, title loan, personal loan, or advance loan',
                          'Money transfers', 'Debt or credit management',
                          'Other financial service', 'Virtual currency'], dtype=object),
              'xaxis': 'x',
              'y': array([2163878,  959581,  567007,  397811,  212459,  206373,  140429,  132976,
                            88116,   86205,   68346,   55626,   31574,   30640,    8333,    5541,
                             5376,    5354,    1081,    1058,      18], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Product by Highest Complaint Volume'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Product'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Number of Complaints'}}}
})

        In the above plot we can see the Number of Complaints w.r.t to the Product by Highest Complaint Volume, where
        we can see for the Product 'Credit reporting, credit repair services, or other personal consumer reports' is the highest Complaint Volume with number 2.163878M

        # Insight 4:  Top 10 Companies by Number of Complaints

In [48]:
top_companies = df1['Company'].value_counts().head(10).reset_index()
top_companies.columns = ['Company', 'Number of Complaints']
fig = px.bar(top_companies, x='Company', y='Number of Complaints', title='Top 10 Companies by Number of Complaints')
fig.update_xaxes(title_text='Company')
fig.update_yaxes(title_text='Number of Complaints')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Company=%{x}<br>Number of Complaints=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['EQUIFAX, INC.', 'TRANSUNION INTERMEDIATE HOLDINGS, INC.',
                          'Experian Information Solutions Inc.',
                          'BANK OF AMERICA, NATIONAL ASSOCIATION', 'WELLS FARGO & COMPANY',
                          'JPMORGAN CHASE & CO.', 'CAPITAL ONE FINANCIAL CORPORATION',
                          'CITIBANK, N.A.', 'SYNCHRONY FINANCIAL', 'Navient Solutions, LLC.'],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1075067,  994517,  904819,  140335,  128430,  119144,   98904,   94213,
                            51041,   39604], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Top 10 Companies by Number of Complaints'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Company'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Number of Complaints'}}}
})

        In the above diagram, we can see those 10 Companies which have highest number of complaints:
        1st is: EQUIFAX, INC.
        2nd is: TRANSUNION INTERMEDIATE HOLDINGS, INC.
        3rd is: Experian Information Solutions Inc. 
        4th is: BANK OF AMERICA, NATIONAL ASSOCIATION 
        5th is: WELLS FARGO & COMPANY and so on

        # Insight 5: Distribution of Complaints by State


In [50]:
state_complaints = df1['State'].value_counts().reset_index()
state_complaints.columns = ['State', 'Number of Complaints']
fig = px.choropleth(state_complaints, locations='State', locationmode='USA-states', color='Number of Complaints', 
                    scope='usa', title='Distribution of Complaints by State', color_continuous_scale='Viridis')
fig.update_layout(geo=dict(bgcolor='rgba(0,0,0,0)', lakecolor='rgba(0,0,0,0)'))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'geo': 'geo',
              'hovertemplate': 'State=%{location}<br>Number of Complaints=%{z}<extra></extra>',
              'locationmode': 'USA-states',
              'locations': array(['FL', 'CA', 'TX', 'GA', 'NY', 'PA', 'IL', 'NJ', 'NC', 'MD', 'OH', 'VA',
                                  'MI', 'AL', 'SC', 'TN', 'AZ', 'LA', 'NV', 'MA', 'MO', 'IN', 'WA', 'CO',
                                  'MS', 'CT', 'WI', 'MN', 'AR', 'OR', 'OK', 'KY', 'DE', 'UT', 'DC', 'KS',
                                  'IA', 'NM', 'PR', 'NE', 'RI', 'HI', 'NH', 'ID', 'WV', 'ME', 'MT', 'AK',
                                  'ND', 'SD', 'VT', 'WY', 'AE', 'VI', 'AP',
                                  'UNITED STATES MINOR OUTLYING ISLANDS', 'GU', 'FM', 'AA', 'MP', 'AS',
                                  'MH', 'PW'], dtype=object),
              'name': '',
              'type': 'choropleth',
              'z': array([664972, 589948, 550630, 354237, 331769, 244915, 215559, 174698, 168306,
                          135396, 129723, 124448, 118768,  95875,  95029,  93903,  91876,  91056,
                           76079,  71934,  66121,  59772,  59561,  52025,  45261,  43783,  40679,
                           39114,  32970,  29233,  27155,  26906,  26878,  23569,  22525,  16895,
                           14982,  14270,  12077,  11643,  11140,  10753,   9429,   9428,   8285,
                            7547,   5026,   4037,   3559,   3512,   3354,   3014,   1293,    973,
                             742,    475,    351,    116,     59,     56,     47,     33,     13],
                         dtype=int64)}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Number of Complaints'}},
                             'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                            '#482878'], [0.2222222222222222,
                                            '#3e4989'], [0.3333333333333333,
                                            '#31688e'], [0.4444444444444444,
                                            '#26828e'], [0.5555555555555556,
                                            '#1f9e89'], [0.6666666666666666,
                                            '#35b779'], [0.7777777777777778,
                                            '#6ece58'], [0.8888888888888888,
                                            '#b5de2b'], [1.0, '#fde725']]},
               'geo': {'bgcolor': 'rgba(0,0,0,0)',
                       'center': {},
                       'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                       'lakecolor': 'rgba(0,0,0,0)',
                       'scope': 'usa'},
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Distribution of Complaints by State'}}
})

        In the above map diagram we can see the 'Distribution of Complaints by State in America' where
        1st state: we can see the state 'FL' means Florida has highest highest number of complaints with number  664.972K
        2nd state: we can see 'CA' means California has second highest number of complaints with number 589.948K
        3rd state: 'TX' means Texas has third highest number of complaints with number 550.63K and so on 

        You can see by hovering and zooming in and out the map above 


        # Insight 6: # Response time for each complaint

In [57]:

df1['Response Time'] = df1['Date sent to company'] - df1['Date received']

average_response_time = df1.groupby('Company response to consumer')['Response Time'].mean()  # Group by 'Company response to consumer' and calculating the mean response time
average_response_time


C:\Users\rames\AppData\Local\Temp\ipykernel_25232\3528040734.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Company response to consumer
Closed                            6 days 07:48:46.239282266
Closed with explanation           1 days 12:21:34.269284513
Closed with monetary relief       2 days 01:45:49.427326567
Closed with non-monetary relief   0 days 13:00:14.879025266
Closed with relief                3 days 19:49:40.995475113
Closed without relief             4 days 10:40:17.730020147
In progress                       0 days 01:12:46.540361911
Untimely response                 6 days 18:05:16.097560975
Name: Response Time, dtype: timedelta64[ns]

        So, on average companies take approximately 6 days to respond and close consumer complaints, with variations based on the nature of the response, 
        ranging from quick resolutions in under a day for non-monetary relief to longer periods, such as over 6 days for untimely responses.


        Insight 7: Distribution of Complaints by Product and Sub-product

In [63]:
product_subproduct = df1.groupby(['Product', 'Sub-product']).size().reset_index()
product_subproduct.columns = ['Product', 'Sub-product', 'Number of Complaints']
fig = px.sunburst(product_subproduct, path=['Product', 'Sub-product'], values='Number of Complaints', title='Distribution of Complaints by Product and Sub-product')

fig.update_xaxes(title_text='Product and Sub-product')
fig.update_yaxes(title_text='Number of Complaints')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'branchvalues': 'total',
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('labels=%{label}<br>Number of C' ... 'nt}<br>id=%{id}<extra></extra>'),
              'ids': array(['Bank account or service/(CD) Certificate of deposit',
                            'Debt collection/Auto', 'Debt collection/Auto debt',
                            'Checking or savings account/CD (Certificate of Deposit)',
                            'Bank account or service/Cashing a check without an account',
                            'Other financial service/Check cashing',
                            'Money transfer, virtual currency, or money service/Check cashing service',
                            'Bank account or service/Checking account',
                            'Checking or savings account/Checking account',
                            'Mortgage/Conventional adjustable mortgage (ARM)',
                            'Mortgage/Conventional fixed mortgage',
                            'Credit reporting, credit repair services, or other personal consumer reports/Conventional home mortgage',
                            'Mortgage/Conventional home mortgage', 'Debt collection/Credit card',
                            'Debt collection/Credit card debt',
                            'Other financial service/Credit repair',
                            'Credit reporting, credit repair services, or other personal consumer reports/Credit repair services',
                            'Debt or credit management/Credit repair services',
                            'Checking or savings account/Credit reporting',
                            'Credit card/Credit reporting',
                            'Credit card or prepaid card/Credit reporting',
                            'Credit reporting/Credit reporting',
                            'Credit reporting or other personal consumer reports/Credit reporting',
                            'Credit reporting, credit repair services, or other personal consumer reports/Credit reporting',
                            'Payday loan/Credit reporting',
                            'Payday loan, title loan, or personal loan/Credit reporting',
                            'Debt or credit management/Debt settlement',
                            'Money transfer, virtual currency, or money service/Debt settlement',
                            'Other financial service/Debt settlement',
                            'Money transfer, virtual currency, or money service/Domestic (US) money transfer',
                            'Money transfers/Domestic (US) money transfer',
                            'Virtual currency/Domestic (US) money transfer',
                            'Payday loan, title loan, personal loan, or advance loan/Earned wage access',
                            'Prepaid card/Electronic Benefit Transfer / EBT card',
                            'Mortgage/FHA mortgage', 'Debt collection/Federal student loan',
                            'Debt collection/Federal student loan debt',
                            'Student loan/Federal student loan servicing',
                            'Money transfer, virtual currency, or money service/Foreign currency exchange',
                            'Other financial service/Foreign currency exchange',
                            'Prepaid card/General purpose card',
                            'Credit card/General-purpose credit card or charge card',
                            'Credit card or prepaid card/General-purpose credit card or charge card',
                            'Credit card or prepaid card/General-purpose prepaid card',
                            'Prepaid card/General-purpose prepaid card',
                            'Credit card or prepaid card/Gift card', 'Prepaid card/Gift card',
                            'Prepaid card/Gift or merchant card',
                            'Credit card or prepai

        Here, we can see the distribution of complaints across different product and sub-product categories, revealing the relative frequency of complaints within each category and their hierarchical relationships. 
        It enables quick identification of the most common complaint categories and their proportions in the dataset.

        You can see the numbers by hovering over the plot

        Insight 8: Most Common Issues Reported

In [64]:
top_issues = df1['Issue'].value_counts().head(10).reset_index()
top_issues.columns = ['Issue', 'Number of Complaints']
fig = px.bar(top_issues, x='Issue', y='Number of Complaints', title='Top 10 Most Common Issues Reported')
fig.update_xaxes(title_text='Issue')
fig.update_yaxes(title_text='Number of Complaints')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Issue=%{x}<br>Number of Complaints=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Incorrect information on your report', 'Improper use of your report',
                          "Problem with a credit reporting company's investigation into an existing problem",
                          'Attempts to collect debt not owed',
                          "Problem with a company's investigation into an existing problem",
                          'Managing an account', 'Loan modification,collection,foreclosure',
                          'Incorrect information on credit report',
                          'Written notification about debt', 'Trouble during payment process'],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1504942,  799797,  589336,  211699,  204610,  129205,  112306,  102684,
                            96690,   81136], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Top 10 Most Common Issues Reported'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Issue'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Number of Complaints'}}}
})

        In above plot, we can see the Top 10 most common issues reported, where
        1st is: Incorrect information on your report,
        2nd is: Improper use of your report,
        3rd is: Problem with a credit reporting company's investigation into an existing problem,
        4th is: Attempts to collect debt not owed and so on

        # Insight 9: Distribution of Complaints by Company Response to Consumer

In [67]:
response_distribution = df1['Company response to consumer'].value_counts().reset_index()
response_distribution.columns = ['Company Response', 'Number of Complaints']
fig = px.pie(response_distribution, values='Number of Complaints', names='Company Response', title='Distribution of Complaints by Company Response to Consumer')
fig.update_layout(title='Distribution of Complaints by Company Response to Consumer')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': 'Company Response=%{label}<br>Number of Complaints=%{value}<extra></extra>',
              'labels': array(['Closed with explanation', 'Closed with non-monetary relief',
                               'In progress', 'Closed with monetary relief', 'Closed without relief',
                               'Closed', 'Untimely response', 'Closed with relief'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'type': 'pie',
              'values': array([3445835, 1297585,  218783,  153316,   17868,   17611,   11480,    5304],
                              dtype=int64)}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Distribution of Complaints by Company Response to Consumer'}}
})

        In above pie graph, we can see the distribution of Complaints by Company Response to Consumer, where we can see
        1st is: 'Closed with explanation' has highest complaints with number 3445835,
        2nd is: 'Closed with non-monetary relief' has 1297585 and so on


        # Insight 10: Distribution of Complaints by Timely Response

In [69]:
timely_response = df1['Timely response?'].value_counts().reset_index()
timely_response.columns = ['Timely Response', 'Number of Complaints']
fig = px.pie(timely_response, values='Number of Complaints', names='Timely Response', title='Distribution of Complaints by Timely Response')
fig.update_layout(title='Distribution of Complaints by Timely Response')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': 'Timely Response=%{label}<br>Number of Complaints=%{value}<extra></extra>',
              'labels': array(['Yes', 'No'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'type': 'pie',
              'values': array([5109230,   58552], dtype=int64)}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Distribution of Complaints by Timely Response'}}
})

        Here, we can see the distribution of Complaints by Timely Response has the proportions as,
        1st: 'Yes' with 98.9% that is 5109230 complaints
        2nd: 'No' with 1.13% that is 58552

# __Q2__


        Given an unsorted array of integers, find the length of the longest continuous increasing subsequence (subarray). 
        Example 1:
        Input: [1,3,5,4,7]
        Output: 3 
        Example 2:
        Input: [2,2,2,2,2]
        Output: 1


In [70]:
def LengthOfLCIS(nums):
    if not nums:
        return 0
    
    maxLength = 1
    currentLength = 1
    
    for i in range(1, len(nums)):       # Iterating through the array
        if nums[i] > nums[i - 1]:
            currentLength += 1
            maxLength = max(maxLength, currentLength)      # Finding max value
        else:
            currentLength = 1    
    return maxLength

# Example 1
nums1 = [1, 3, 5, 4, 7]
print("Output: ", LengthOfLCIS(nums1))  # Output: 3

# Example 2
nums2 = [2, 2, 2, 2, 2]
print("Output: ", LengthOfLCIS(nums2))  # Output: 1


Output:  3
Output:  1


# __Q3__


        Given a list of non negative integers, arrange them such that they form the largest number.
 
        Example 1:
        Input: [10,2]
        Output: "210"
        Example 2:
 
        Input: [3,30,34,5,9]
        Output: "9534330"



In [7]:
from typing import List

def largestNumber(nums: List[int]) -> str:
    nums = list(map(str, nums))     # Converting integers to strings for comparison
    nums.sort(key=lambda x: x * 10 if x == "0" else x, reverse=True)      # Defining comparator for sorting
    return "".join(nums)       # Concatenating the sorted strings

nums1 = [10, 2]
print(largestNumber(nums1))  # Output: "210"

nums2 = [3, 30, 34, 5, 9]
print(largestNumber(nums2))  # Output: "9534330"


210
9534303


# __Q4__

        Store all the "servlet-name", and "servlet-class" to a csv file from the attached sample_json.json file using Python.

In [4]:
import pandas as pd
import json

with open('DT A1 sample_json (1) (1).json') as json_file:       # Loading JSON data from sample.json.json file
    data = json.load(json_file)

servlet_data = data["web-app"]["servlet"]             # Extracting servlet data

# Initializing lists to store servlet-name and servlet-class
servlet_names = []
servlet_classes = []

# Extracting servlet-name and servlet-class
for servlet in servlet_data:
    servlet_names.append(servlet["servlet-name"])
    servlet_classes.append(servlet["servlet-class"])

# Creating DataFrame with column names and values
df = pd.DataFrame({
    'servlet-name': servlet_names,
    'servlet-class': servlet_classes
})

# Saving DataFrame to CSV
df.to_csv('servlet_data.csv', index=False)

print("CSV file created successfully!")


CSV file created successfully!


        By Samit Dhawal